In [25]:
import numpy as np
import torch
import walnut

In [26]:
dim = (100, 3, 5, 5)

In [27]:
n_x = np.random.randint(0, 255, dim)
t_x = torch.nn.Parameter(torch.from_numpy(n_x).float(), requires_grad=True)

# Forward
### NumpyNN

In [28]:
from walnut.nn import layers

ln = layers.Layernorm(input_shape=n_x.shape[1:])
ln.compile()
ln.x = walnut.Tensor(n_x)

In [29]:
ln.forward()
ln.y

tnsor([[[[ 0.6724,  1.4929,  1.8445, -0.2653, -0.9099],
         [-0.7048,  1.947 , -0.0162, -1.6132, -1.496 ],
         [ 0.9508,  1.361 , -0.1774, -1.2616,  0.7896],
         [-1.5399, -0.1774, -0.6462,  0.9361, -0.2067],
         [ 0.8629,  1.3903,  1.2291,  0.8629,  0.2768]],

        [[ 0.8482, -0.7195,  0.5405, -1.2469,  0.0131],
         [ 0.731 , -0.2799, -0.3385,  0.4673, -1.6132],
         [-1.6425, -1.0711,  0.101 , -0.866 ,  0.9361],
         [-0.3239,  0.5112, -0.4997, -0.7048, -0.7048],
         [ 0.7456, -0.2067,  0.8922, -1.3202, -0.0895]],

        [[ 0.2622, -0.529 ,  0.8482, -1.6278,  0.4966],
         [ 0.8043,  1.9324,  0.1596, -1.4667, -1.1883],
         [-1.5985,  1.4929,  1.1852,  1.7712, -1.2469],
         [-0.1334, -0.3239, -0.5144,  0.3061,  1.2877],
         [ 0.2475, -0.6755, -1.2909,  0.2768, -0.236 ]]],


       [[[-1.0511,  1.0863,  0.1946, -0.1988,  0.1159],
         [-1.2478, -1.6018, -1.4838,  1.1912,  1.3354],
         [ 1.4403,  0.129 ,  1.0994,  0.

### Torch

In [30]:
t_eps = 1e-7

gbshape = t_x.shape[1:]
t_gamma = torch.ones(gbshape, requires_grad=True)
t_beta = torch.zeros(gbshape, requires_grad=True)

In [31]:
std_axis = tuple(i + 1 for i in range(t_x.ndim - 1))
t_mean = t_x.mean(axis=std_axis, keepdims=True)
t_var = t_x.var(axis=std_axis, keepdims=True)
t_xhat = (t_x - t_mean) / torch.sqrt(t_var + t_eps)
t_y = t_gamma * t_xhat + t_beta
t_y

tensor([[[[ 0.6724,  1.4929,  1.8445, -0.2653, -0.9099],
          [-0.7048,  1.9470, -0.0162, -1.6132, -1.4960],
          [ 0.9508,  1.3610, -0.1774, -1.2616,  0.7896],
          [-1.5399, -0.1774, -0.6462,  0.9361, -0.2067],
          [ 0.8629,  1.3903,  1.2291,  0.8629,  0.2768]],

         [[ 0.8482, -0.7195,  0.5405, -1.2469,  0.0131],
          [ 0.7310, -0.2799, -0.3385,  0.4673, -1.6132],
          [-1.6425, -1.0711,  0.1010, -0.8660,  0.9361],
          [-0.3239,  0.5112, -0.4997, -0.7048, -0.7048],
          [ 0.7456, -0.2067,  0.8922, -1.3202, -0.0895]],

         [[ 0.2622, -0.5290,  0.8482, -1.6278,  0.4966],
          [ 0.8043,  1.9324,  0.1596, -1.4667, -1.1883],
          [-1.5985,  1.4929,  1.1852,  1.7712, -1.2469],
          [-0.1334, -0.3239, -0.5144,  0.3061,  1.2877],
          [ 0.2475, -0.6755, -1.2909,  0.2768, -0.2360]]],


        [[[-1.0511,  1.0863,  0.1946, -0.1988,  0.1159],
          [-1.2478, -1.6018, -1.4838,  1.1912,  1.3354],
          [ 1.4403,  0.

# Backward

In [32]:
n_dy = np.random.randn(*dim)
t_dy = torch.nn.Parameter(torch.from_numpy(n_dy).float())

In [33]:
ln.y.grad = n_dy
ln.backward()
t_y.backward(t_dy)

### Beta

In [34]:
ln.b.grad

array([[[ -6.8745,   2.5282,  -7.5034,  -2.5658, -19.0653],
        [ -3.828 ,  15.1498,  -8.0293,  17.7866,  -9.483 ],
        [  4.2992,  -1.383 ,   3.9966, -12.3919,   5.7105],
        [ -5.1272,   1.4636,  10.5907,  -4.3506,  11.9166],
        [ -0.1676,  -6.344 ,  18.6541,   1.4524, -13.6144]],

       [[  4.8318,  10.9558,  13.8462,  12.1537,  -5.5048],
        [ -7.9963,  15.5748,   4.5651, -16.2047,  18.8738],
        [  4.6598,   1.8884,  -2.5904,  -1.2032,   7.8903],
        [ -5.8022,   0.8267,  -0.556 ,   2.3177,   5.7754],
        [ -9.9095,  13.8593,   1.0216,   3.2008,  -2.9815]],

       [[ 13.3806,  -9.2322,  -6.6593,  -4.5233,  16.5974],
        [ 11.276 , -12.0174,  -4.1738, -23.5248, -13.0714],
        [ -6.9915,   5.1012,   8.0202,  -0.429 ,   8.1217],
        [ -6.9013, -12.3207,  -9.0075,  -4.4087,   6.9144],
        [ -6.8828,   2.1677,  -8.194 ,   3.4165,  -3.9691]]])

In [35]:
t_beta.grad

tensor([[[ -6.8745,   2.5282,  -7.5034,  -2.5658, -19.0653],
         [ -3.8280,  15.1498,  -8.0293,  17.7866,  -9.4830],
         [  4.2992,  -1.3830,   3.9966, -12.3919,   5.7105],
         [ -5.1272,   1.4636,  10.5907,  -4.3506,  11.9166],
         [ -0.1676,  -6.3440,  18.6541,   1.4524, -13.6144]],

        [[  4.8318,  10.9558,  13.8462,  12.1537,  -5.5048],
         [ -7.9963,  15.5748,   4.5651, -16.2047,  18.8738],
         [  4.6598,   1.8884,  -2.5904,  -1.2032,   7.8903],
         [ -5.8022,   0.8267,  -0.5560,   2.3177,   5.7754],
         [ -9.9095,  13.8593,   1.0216,   3.2008,  -2.9815]],

        [[ 13.3806,  -9.2322,  -6.6593,  -4.5233,  16.5974],
         [ 11.2760, -12.0174,  -4.1738, -23.5248, -13.0714],
         [ -6.9915,   5.1012,   8.0202,  -0.4290,   8.1217],
         [ -6.9013, -12.3207,  -9.0075,  -4.4087,   6.9144],
         [ -6.8828,   2.1677,  -8.1940,   3.4165,  -3.9691]]])

### Gamma

In [37]:
ln.w.grad

array([[[  1.6174,   0.8843,   6.0745,  -2.1899,   0.5838],
        [ -1.2576, -19.1244,   2.4249,  -3.2157,  -1.5622],
        [ -5.3376, -18.0982, -17.8271,  18.4721,  -4.3923],
        [  1.2169,  -2.2855,   8.7582,   4.0655,  -7.8449],
        [ -8.8402,   4.3148,  -7.6438,  -0.6694, -14.2739]],

       [[ -6.4695,   3.9183,   5.6816, -11.269 ,   8.709 ],
        [  7.0403,  -1.9037,  -0.7802,  -9.1801,  29.1784],
        [  3.585 ,   1.767 ,  -6.3517,  -6.403 ,  -2.8004],
        [ -8.5662,  11.0958,   4.4134, -13.5449,   2.6728],
        [ -7.7715,  -3.9314,  -1.0569,  14.5737, -10.9101]],

       [[ -7.2271,   0.9481,  21.8799,  -2.6851,  -8.8259],
        [ 15.7645,  -8.3086,   1.4507,   2.6237,  -7.9564],
        [-12.5582,   4.0568,  -5.0695,  20.8969, -14.8906],
        [ -6.2894,  11.0757,  -7.1834,  19.1862,   9.6979],
        [ 16.9774,  -9.7846,   5.0377,  -5.1461,  -0.2413]]])

In [38]:
t_gamma.grad

tensor([[[  1.6174,   0.8843,   6.0745,  -2.1899,   0.5838],
         [ -1.2576, -19.1244,   2.4249,  -3.2157,  -1.5622],
         [ -5.3376, -18.0982, -17.8271,  18.4721,  -4.3923],
         [  1.2169,  -2.2855,   8.7582,   4.0655,  -7.8449],
         [ -8.8402,   4.3148,  -7.6438,  -0.6694, -14.2739]],

        [[ -6.4695,   3.9183,   5.6816, -11.2690,   8.7090],
         [  7.0403,  -1.9037,  -0.7802,  -9.1801,  29.1784],
         [  3.5850,   1.7670,  -6.3517,  -6.4030,  -2.8004],
         [ -8.5662,  11.0958,   4.4134, -13.5449,   2.6728],
         [ -7.7715,  -3.9314,  -1.0569,  14.5737, -10.9101]],

        [[ -7.2271,   0.9481,  21.8799,  -2.6851,  -8.8259],
         [ 15.7645,  -8.3086,   1.4507,   2.6237,  -7.9564],
         [-12.5582,   4.0568,  -5.0695,  20.8969, -14.8906],
         [ -6.2894,  11.0757,  -7.1834,  19.1862,   9.6979],
         [ 16.9774,  -9.7846,   5.0377,  -5.1461,  -0.2413]]])

### X

In [39]:
ln.x.grad

array([[[[ 2.0399e-03,  5.7757e-03,  1.3192e-02, -1.2987e-03,
           2.5975e-02],
         [ 2.1170e-02,  2.8197e-04, -3.1219e-03,  9.7828e-03,
          -1.7499e-03],
         [-3.6623e-03,  9.6910e-03,  8.3913e-03,  1.1800e-02,
           1.1971e-02],
         [ 7.4735e-03, -2.2421e-02, -8.0274e-03, -4.5349e-03,
           1.2387e-02],
         [-1.6583e-02, -7.4239e-03, -1.1290e-02,  1.3081e-02,
          -8.0601e-03]],

        [[-1.9620e-03,  1.2429e-03,  9.1367e-03,  1.8097e-02,
          -1.9986e-02],
         [-1.2654e-03, -3.4792e-02,  5.7747e-03,  9.8504e-03,
          -2.6722e-02],
         [-7.1790e-03,  2.9832e-02, -9.1373e-03,  2.6632e-02,
          -3.1187e-02],
         [ 1.7308e-02, -1.3878e-02, -3.3859e-02, -1.5197e-02,
          -2.1945e-02],
         [ 4.2228e-03,  3.3952e-02, -4.7587e-03, -1.2558e-02,
           2.0013e-03]],

        [[ 2.3712e-02, -1.3229e-02, -6.3494e-03, -2.4184e-02,
          -7.3554e-03],
         [-5.1699e-03,  7.6557e-03, -1.8084e-04, -

In [40]:
t_x.grad

tensor([[[[ 2.0399e-03,  5.7757e-03,  1.3192e-02, -1.2987e-03,  2.5975e-02],
          [ 2.1170e-02,  2.8197e-04, -3.1219e-03,  9.7828e-03, -1.7499e-03],
          [-3.6623e-03,  9.6910e-03,  8.3913e-03,  1.1800e-02,  1.1971e-02],
          [ 7.4735e-03, -2.2421e-02, -8.0274e-03, -4.5349e-03,  1.2387e-02],
          [-1.6583e-02, -7.4239e-03, -1.1290e-02,  1.3081e-02, -8.0601e-03]],

         [[-1.9620e-03,  1.2429e-03,  9.1367e-03,  1.8097e-02, -1.9986e-02],
          [-1.2654e-03, -3.4792e-02,  5.7747e-03,  9.8504e-03, -2.6722e-02],
          [-7.1790e-03,  2.9832e-02, -9.1373e-03,  2.6632e-02, -3.1187e-02],
          [ 1.7308e-02, -1.3878e-02, -3.3859e-02, -1.5197e-02, -2.1945e-02],
          [ 4.2228e-03,  3.3952e-02, -4.7587e-03, -1.2558e-02,  2.0013e-03]],

         [[ 2.3712e-02, -1.3229e-02, -6.3494e-03, -2.4184e-02, -7.3554e-03],
          [-5.1699e-03,  7.6557e-03, -1.8084e-04, -4.6316e-03, -6.2252e-03],
          [ 2.2261e-03,  1.3032e-02, -1.0379e-02,  4.9295e-03,  1.5956e-